In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [5]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,d*24:(d+1)*24]=train_data1.iloc[m*20*18+d*18:m*20*18+(d+1)*18,:]
    year_data[m]=month_data

In [6]:
year_data1=year_data.copy()
month_list=[]
for i in range(12):
    month_list.append(pd.DataFrame(year_data1[i]))
all_train_data1=pd.concat(month_list,axis=1).T

In [7]:
all_train_data1.iloc[:,0].mean()

22.527413194444442

In [8]:
all_train_data1.T.shape

(18, 5760)

In [9]:
# temperature always >=0 in Taiwan and change fast when temperature <=0 ,so correct it
for idx in [0]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <=0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
---------- pm2.5 start : 
 -3.1 

-8.0
-7.2
-6.8
-6.5
-7.1
-7.4
-8.1
-8.3
-8.4
-9.3
-10.6
-11.2
-12.1
0.0
0.0
0.0
0.0
-12.3
pm2.5 end ------------
correct value between  : 19.0 18.0
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11


In [10]:
for idx in [1,2,4,6,9,12,13]:	
	for m in range(12):
		print(' month : ',m)
		i=0
		while i<480:
			if year_data1[m][idx,i] <0:
				print('---------- pm2.5 start : \n',year_data1[m][idx,i],'\n')
				for k in range(30):
					if k+i+1>479:
						break
					else:
						if year_data1[m][idx,k+i+1] >=0:
							break
						else:
							print(year_data1[m][idx,k+i+1])
				i=i+k
				print('pm2.5 end ------------')
				print('correct value between  :',year_data1[m][idx,i-1-k],year_data1[m][idx,i+1])
				## add correct to mean value
				year_data1[m][idx,i-k:i+1]=(year_data1[m][idx,i-1-k]+year_data1[m][idx,i+1])/2
			i=i+1

 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.2 

-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
-0.2
pm2.5 end ------------
correct value between  : 1.8 1.8
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -0.12 

pm2.5 end ------------
correct value between  : 0.34 0.26
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
 month :  5
 month :  6
 month :  7
 month :  8
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 2.5 0.3
---------- pm2.5 start : 
 -0.1 

pm2.5 end ------------
correct value between  : 0.1 1.0
 month :  9
 month :  10
 month :  11
---------- pm2.5 start : 
 -1.1 

pm2.5 end ------------
correct value between  : 6.6 1.2
 month :  0
 month :  1
 month :  2
 month :  3
 month :  4
---------

In [11]:
all_train_data1.iloc[0,:5]

0    14.00
1     1.80
2     0.51
3     0.20
4     0.90
Name: 0, dtype: float64

In [12]:
all_train_data2=all_train_data1.apply(lambda x : (x-x.mean())/x.std())

In [13]:
all_train_data1.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,22.527413,1.702396,0.388363,0.140427,2.135729,10.125990,12.247726,31.905469,42.709201,21.414236,0.200625,73.229167,2.763125,1.839653,156.329271,158.482795,2.297240,1.712760
std,6.290152,0.125265,0.323573,0.104645,2.282155,6.187555,7.577133,18.703486,26.222292,16.662537,2.045443,13.361351,1.816940,0.181839,95.745881,94.697432,1.065408,1.062683
min,-12.300000,-0.200000,-0.120000,0.000000,-1.100000,0.000000,-2.400000,0.000000,0.000000,-1.000000,0.000000,29.000000,-1.600000,-0.200000,0.100000,0.000000,0.000000,0.000000
25%,18.000000,1.600000,0.250000,0.070000,1.100000,5.900000,7.300000,18.000000,24.000000,9.000000,0.000000,64.000000,1.600000,1.700000,72.000000,76.000000,1.500000,0.900000
50%,23.000000,1.700000,0.340000,0.110000,1.600000,8.600000,10.000000,29.000000,38.000000,18.000000,0.000000,75.000000,2.300000,1.800000,119.000000,121.500000,2.100000,1.500000
75%,27.000000,1.800000,0.450000,0.180000,2.300000,13.000000,15.000000,42.000000,57.000000,29.250000,0.000000,84.000000,3.400000,1.900000,213.000000,219.000000,2.900000,2.300000
max,36.000000,2.000000,7.570000,1.300000,31.000000,46.000000,71.000000,231.000000,181.000000,112.000000,74.000000,99.000000,22.000000,3.000000,360.000000,360.000000,7.700000,7.000000


In [14]:
all_train_data1.iloc[:,0].mean()

22.527413194444442

In [15]:
all_train_data2.iloc[0:10,0]

0   -1.355677
1   -1.355677
2   -1.355677
3   -1.514655
4   -1.673634
5   -1.673634
6   -1.673634
7   -1.673634
8   -1.196698
9   -0.878741
Name: 0, dtype: float64

In [16]:
all_train_data2.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03,5.760000e+03
mean,2.921351e-15,-4.132630e-13,-1.393800e-14,5.078230e-15,3.474738e-15,-2.338465e-15,-3.438183e-15,-4.431640e-16,2.485088e-16,1.855094e-16,4.130921e-15,-9.199431e-16,5.026265e-16,3.205650e-13,8.574353e-17,5.245418e-16,5.851993e-16,1.040902e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-5.536816e+00,-1.518692e+01,-1.571093e+00,-1.341940e+00,-1.417839e+00,-1.636509e+00,-1.933149e+00,-1.705857e+00,-1.628736e+00,-1.345187e+00,-9.808389e-02,-3.310232e+00,-2.401358e+00,-1.121682e+01,-1.631707e+00,-1.673570e+00,-2.156207e+00,-1.611732e+00
25%,-7.197621e-01,-8.174309e-01,-4.276098e-01,-6.730107e-01,-4.538382e-01,-6.829821e-01,-6.529812e-01,-7.434694e-01,-7.134846e-01,-7.450388e-01,-9.808389e-02,-6.907360e-01,-6.401558e-01,-7.680033e-01,-8.807613e-01,-8.710141e-01,-7.482952e-01,-7.648192e-01
50%,7.513122e-02,-1.912605e-02,-1.494652e-01,-2.907653e-01,-2.347470e-01,-2.466224e-01,-2.966459e-01,-1.553437e-01,-1.795877e-01,-2.049049e-01,-9.808389e-02,1.325340e-01,-2.548928e-01,-2.180656e-01,-3.898786e-01,-3.905364e-01,-1.851306e-01,-2.002106e-01
75%,7.110459e-01,7.791788e-01,1.904894e-01,3.781641e-01,7.198057e-02,4.644824e-01,3.632342e-01,5.397139e-01,5.449866e-01,4.702623e-01,-9.808389e-02,8.061186e-01,3.505206e-01,3.318721e-01,5.918869e-01,6.390586e-01,5.657555e-01,5.526008e-01
max,2.141854e+00,2.375788e+00,2.219482e+01,1.108104e+01,1.264781e+01,5.797768e+00,7.753892e+00,1.064478e+01,5.273788e+00,5.436493e+00,3.607990e+01,1.928760e+00,1.058751e+01,6.381187e+00,2.127201e+00,2.128011e+00,5.071072e+00,4.975368e+00


In [17]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

In [18]:
x=np.empty((471*12,18*9))
y=np.empty((471*12,1))

In [19]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=all_train_data2.iloc[m*480+g:m*480+g+9,:].values.reshape(1,-1)
        y[m*471+g:g*471+g+1,:]=all_train_data2.iloc[m*480+g+9:m*480+g+9+1,9].values.reshape(1,1)

In [20]:
x[9,9:11]

array([ 0.5152735 , -0.09808389])

In [21]:
y[0:2]

array([[0.5152735 ],
       [1.17543706]])

In [22]:
all_train_data2.iloc[10:11,9]

10    1.175437
Name: 9, dtype: float64

In [23]:
x[1,-10:]

array([-0.17958771,  0.5152735 , -0.09808389, -1.28947789,  6.73487971,
        0.88180982, -1.15231349, -0.05789803, -1.40532054, -1.32942776])

In [24]:
y[0]

array([0.5152735])

In [25]:
kf=KFold(n_splits=10,shuffle=True,random_state=42)

def rmse(y,y_hat):
    return np.sqrt(mean_squared_error(y,y_hat))

def cv_rmse(model,x=x,y=y):
    rmse=np.sqrt( -cross_val_score(model,x,y,scoring='neg_mean_squared_error',cv=kf) )
    return rmse

In [26]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [27]:
ridge=make_pipeline(RobustScaler(),RidgeCV(alphas=alphas_alt,cv=kf))
lasso=make_pipeline(RobustScaler(),LassoCV(max_iter=1e7,alphas=alphas2,random_state=42,cv=kf))
elasticnet=make_pipeline(RobustScaler(),ElasticNetCV(max_iter=1e7,alphas=e_alphas,cv=kf,l1_ratio=e_l1ratio))

In [28]:
score=cv_rmse(ridge)
print("Ridge: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge: 0.3776 (0.0248)



In [29]:
score = cv_rmse(lasso)
print("LASSO: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d arr

LASSO: 0.3759 (0.0246)



In [30]:
score = cv_rmse(elasticnet)
print("elastic net: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d arr

elastic net: 0.3760 (0.0247)



In [31]:
############ linear model with  kfold

In [30]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [31]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [32]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [33]:
x11=torch.Tensor(x)
y11=torch.Tensor(y)

In [34]:
def get_net():
    return nn.Sequential(nn.Linear(x11.shape[1],128),nn.ReLU(),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x11,y11,256,0,0)

epoch = 1 train_loss : 0.371675 , test loss : 0.412721
epoch = 2 train_loss : 0.296409 , test loss : 0.353679
epoch = 3 train_loss : 0.249648 , test loss : 0.307367
epoch = 4 train_loss : 0.219696 , test loss : 0.278300
epoch = 5 train_loss : 0.196267 , test loss : 0.251929
epoch = 6 train_loss : 0.174817 , test loss : 0.233486
epoch = 7 train_loss : 0.160651 , test loss : 0.218536
epoch = 8 train_loss : 0.147730 , test loss : 0.207120
epoch = 9 train_loss : 0.138490 , test loss : 0.199888
epoch = 10 train_loss : 0.131910 , test loss : 0.198027
epoch = 11 train_loss : 0.121803 , test loss : 0.188596
epoch = 12 train_loss : 0.111415 , test loss : 0.180612
epoch = 13 train_loss : 0.107153 , test loss : 0.179309
epoch = 15 train_loss : 0.095786 , test loss : 0.172003
epoch = 16 train_loss : 0.091395 , test loss : 0.171546
epoch = 17 train_loss : 0.088295 , test loss : 0.168564
epoch = 19 train_loss : 0.081826 , test loss : 0.168512
---------------------------------------------------------

In [37]:
0.15*16+22

24.4

### change y to original value

In [35]:
for m in range(12):
    for g in range(471):
        x[m*471+g:m*471+g+1,:]=all_train_data2.iloc[m*480+g:m*480+g+9,:].values.reshape(1,-1)
        y[m*471+g:g*471+g+1,:]=all_train_data1.iloc[m*480+g+9:m*480+g+9+1,9].values.reshape(1,1)

In [37]:
x11=torch.Tensor(x)
y11=torch.Tensor(y)

In [43]:
def get_net():
    return nn.Sequential(nn.Linear(x11.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.0005,10,x11,y11,256,0,0)

epoch = 1 train_loss : 647.951294 , test loss : 670.700134
epoch = 2 train_loss : 595.591187 , test loss : 617.502502
epoch = 3 train_loss : 570.112244 , test loss : 594.117126
epoch = 4 train_loss : 552.577759 , test loss : 578.583496
epoch = 5 train_loss : 536.613098 , test loss : 564.098938
epoch = 6 train_loss : 523.258240 , test loss : 551.711365
epoch = 7 train_loss : 510.037598 , test loss : 539.422058
epoch = 8 train_loss : 496.708099 , test loss : 526.543091
epoch = 9 train_loss : 483.165009 , test loss : 512.768433
epoch = 10 train_loss : 468.881012 , test loss : 499.001923
epoch = 11 train_loss : 453.585754 , test loss : 482.989655
epoch = 12 train_loss : 437.246033 , test loss : 466.660889
epoch = 13 train_loss : 419.737183 , test loss : 449.412598
epoch = 14 train_loss : 401.414154 , test loss : 429.527161
epoch = 15 train_loss : 381.792816 , test loss : 410.705444
epoch = 16 train_loss : 361.618958 , test loss : 390.580658
epoch = 17 train_loss : 340.656891 , test loss : 

epoch = 3 train_loss : 574.041565 , test loss : 568.368225
epoch = 4 train_loss : 555.870178 , test loss : 556.936096
epoch = 5 train_loss : 540.556030 , test loss : 547.863464
epoch = 6 train_loss : 526.341370 , test loss : 535.954041
epoch = 7 train_loss : 512.665710 , test loss : 525.398865
epoch = 8 train_loss : 499.021301 , test loss : 513.670959
epoch = 9 train_loss : 485.039978 , test loss : 501.140442
epoch = 10 train_loss : 470.489777 , test loss : 487.778473
epoch = 11 train_loss : 454.877686 , test loss : 474.482300
epoch = 12 train_loss : 438.478363 , test loss : 458.559082
epoch = 13 train_loss : 421.073364 , test loss : 443.254578
epoch = 14 train_loss : 402.579926 , test loss : 425.741272
epoch = 15 train_loss : 383.007904 , test loss : 407.902618
epoch = 16 train_loss : 362.541656 , test loss : 388.541412
epoch = 17 train_loss : 341.669250 , test loss : 369.038849
epoch = 18 train_loss : 320.644531 , test loss : 349.121094
epoch = 19 train_loss : 299.204681 , test loss 

epoch = 75 train_loss : 38.458519 , test loss : 38.473949
epoch = 78 train_loss : 38.471119 , test loss : 38.243309
epoch = 80 train_loss : 38.483303 , test loss : 38.006290
epoch = 85 train_loss : 38.167805 , test loss : 37.562824
epoch = 87 train_loss : 38.073360 , test loss : 37.562386
epoch = 94 train_loss : 38.065800 , test loss : 37.557026
epoch = 98 train_loss : 38.080540 , test loss : 37.528099
epoch = 99 train_loss : 37.961571 , test loss : 37.164780
epoch = 104 train_loss : 37.820011 , test loss : 36.914581
epoch = 115 train_loss : 37.673161 , test loss : 36.786797
epoch = 118 train_loss : 37.825703 , test loss : 36.704365
epoch = 142 train_loss : 37.583252 , test loss : 36.246109
epoch = 175 train_loss : 37.595676 , test loss : 36.153908
epoch = 259 train_loss : 37.354218 , test loss : 36.037594
epoch = 345 train_loss : 37.425079 , test loss : 35.991837
epoch = 404 train_loss : 37.452896 , test loss : 35.948242
----------------------------------------------------------------

epoch = 86 train_loss : 38.221745 , test loss : 35.841839
--------------------------------------------------------------------------
fold 6,train loss mean : 38.221745,test loss : 35.841839
-------------------------------------------------------------------------
epoch = 1 train_loss : 653.523376 , test loss : 692.618652
epoch = 2 train_loss : 597.747742 , test loss : 630.544861
epoch = 3 train_loss : 568.851562 , test loss : 597.138245
epoch = 4 train_loss : 551.172363 , test loss : 576.410400
epoch = 5 train_loss : 535.735779 , test loss : 560.827759
epoch = 6 train_loss : 520.988770 , test loss : 545.165955
epoch = 7 train_loss : 507.511169 , test loss : 531.558228
epoch = 8 train_loss : 493.415131 , test loss : 516.501404
epoch = 9 train_loss : 478.926392 , test loss : 503.504272
epoch = 10 train_loss : 463.891113 , test loss : 488.369293
epoch = 11 train_loss : 447.972656 , test loss : 472.462006
epoch = 12 train_loss : 430.980194 , test loss : 455.516541
epoch = 13 train_loss : 4

epoch = 108 train_loss : 37.919186 , test loss : 35.327553
epoch = 114 train_loss : 37.950809 , test loss : 35.117809
epoch = 218 train_loss : 37.633755 , test loss : 35.091339
epoch = 251 train_loss : 37.625908 , test loss : 34.822033
--------------------------------------------------------------------------
fold 8,train loss mean : 37.625908,test loss : 34.822033
-------------------------------------------------------------------------
epoch = 1 train_loss : 659.456482 , test loss : 645.546265
epoch = 2 train_loss : 602.729431 , test loss : 594.427917
epoch = 3 train_loss : 572.437561 , test loss : 569.404602
epoch = 4 train_loss : 554.675598 , test loss : 555.461365
epoch = 5 train_loss : 539.395630 , test loss : 542.482056
epoch = 6 train_loss : 525.214600 , test loss : 530.284485
epoch = 7 train_loss : 512.018127 , test loss : 517.945740
epoch = 8 train_loss : 498.368500 , test loss : 505.842590
epoch = 9 train_loss : 484.646606 , test loss : 491.020294
epoch = 10 train_loss : 470

epoch = 75 train_loss : 38.328602 , test loss : 40.237507
epoch = 79 train_loss : 38.098667 , test loss : 40.084515
epoch = 80 train_loss : 38.153839 , test loss : 40.043186
epoch = 81 train_loss : 38.179031 , test loss : 40.032951
epoch = 82 train_loss : 38.235321 , test loss : 39.957638
epoch = 84 train_loss : 37.884460 , test loss : 39.749493
epoch = 95 train_loss : 37.798962 , test loss : 39.618126
epoch = 98 train_loss : 37.622265 , test loss : 39.347126
epoch = 107 train_loss : 37.643524 , test loss : 39.291512
epoch = 116 train_loss : 37.549572 , test loss : 39.266895
epoch = 117 train_loss : 37.382343 , test loss : 39.229248
epoch = 118 train_loss : 37.418674 , test loss : 39.168411
epoch = 125 train_loss : 37.308880 , test loss : 38.930038
epoch = 143 train_loss : 37.282761 , test loss : 38.818230
epoch = 166 train_loss : 37.287483 , test loss : 38.710926
epoch = 210 train_loss : 37.120750 , test loss : 38.553799
epoch = 333 train_loss : 37.171101 , test loss : 38.524731
-----

### LSTM with kfold

In [38]:
class LSTM_Reg(nn.Module):
    def __init__(self,input_size,hidden_size,layer_size,output_size):
        super().__init__()
        self.lstm_net=nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=layer_size)
        self.dense=nn.Linear(hidden_size,output_size)
        self.state=None
    
    def forward(self,x,state):
        y,self.state=self.lstm_net(x,state)
        output=self.dense(y.view(-1,y.shape[-1]))
        return output,self.state


In [64]:
def train_k_fold_data_lstm(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    train_l,test_l=[],[]
    state=None
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        test_loss_list,train_loss_list=[],[]
        net.train()
        for x,y in data_iter:
            if state is not None:
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
                    
            x=x.unsqueeze(dim=0)
            (output,state)=net(x,state)
            
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_loss_list.append(l.item())
#       if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        
        with torch.no_grad():
#             dataset_train=torch.utils.data.TensorDataset(train_features,train_labels)
#             data_iter_train=torch.utils.data.DataLoader(dataset_train,batch_size,shuffle=True,drop_last=True)
            
            dataset_test=torch.utils.data.TensorDataset(test_features,test_labels)
            data_iter_test=torch.utils.data.DataLoader(dataset_test,batch_size,shuffle=True,drop_last=True)
            
            for xx_test,yy_test in data_iter_test:
                xx_test=xx_test.unsqueeze(dim=0)
                (output_test,state)=net(xx_test,state)
                test_loss_list.append(loss(output_test,yy_test).item())
            test_loss=np.mean(test_loss_list)

            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
#                 for xx_train,yy_train in data_iter_train:
#                     xx_train=xx_train.unsqueeze(dim=0)
#                     (output_train,state)=net(xx_train,state)
#                     train_loss_list.append(loss(output_train,yy_train).item())

                train_loss=np.mean(train_loss_list)
                
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l

In [65]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [66]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data_lstm(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [ ]:
x9=torch.Tensor(x)
y9=torch.Tensor(y)
def get_net():
    return LSTM_Reg(162,256,2,1)
net1=get_net()
train_kfold(net1,1000,0.001,5,x9,y9,256,0,0)

epoch = 1 train_loss : 0.657967 , test loss : 0.472880
epoch = 2 train_loss : 0.372705 , test loss : 0.378358
epoch = 3 train_loss : 0.284610 , test loss : 0.305523
epoch = 4 train_loss : 0.236541 , test loss : 0.244333
epoch = 5 train_loss : 0.205396 , test loss : 0.215072
epoch = 7 train_loss : 0.158358 , test loss : 0.206296
epoch = 8 train_loss : 0.143984 , test loss : 0.198946
epoch = 9 train_loss : 0.135191 , test loss : 0.191424
epoch = 10 train_loss : 0.125332 , test loss : 0.189128
epoch = 12 train_loss : 0.110688 , test loss : 0.186612
epoch = 13 train_loss : 0.104582 , test loss : 0.157756
epoch = 452 train_loss : 0.001220 , test loss : 0.156650
epoch = 582 train_loss : 0.000930 , test loss : 0.148020
--------------------------------------------------------------------------
fold 1,train loss mean : 0.000930,test loss : 0.148020
-------------------------------------------------------------------------
epoch = 1 train_loss : 0.656729 , test loss : 0.459039
epoch = 2 train_los

In [ ]:
################################

#### LSTM

In [59]:
class LSTM_Reg(nn.Module):
    def __init__(self,input_size,hidden_size,layer_size,output_size):
        super().__init__()
        self.lstm_net=nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=layer_size)
        self.dense=nn.Linear(hidden_size,output_size)
        self.state=None
    
    def forward(self,x,state):
        y,self.state=self.lstm_net(x,state)
        output=self.dense(y.view(-1,y.shape[-1]))
        return output,self.state
    

In [60]:
x9=torch.Tensor(x)
y9=torch.Tensor(y)

In [61]:
a=torch.utils.data.TensorDataset(x9,y9)
for x2,y2 in  torch.utils.data.DataLoader(a,32,shuffle=True):
    print(x2.shape)
    print(y2.shape)
    print(x2.unsqueeze(dim=0).shape)

torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 162])
torch.Size([32, 162])
torch.Size([32, 1])
torch.Size([1, 32, 1

In [62]:
def train_pytorch(num_epochs,lr,train_features,train_labels,batch_size):
    net=LSTM_Reg(162,256,2,1)
    loss=nn.MSELoss()
    optimizer=optim.Adam(net.parameters(),lr=5e-3)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True,drop_last=True)
    state=None
    for e in range(num_epochs):
        l_sum=[]
        net.train()
        for x,y in data_iter:
            if state is not None:
                if isinstance(state ,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
                
            x=x.unsqueeze(dim=0)
            (output,state)=net(x,state)
            l=loss(output,y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            l_sum.append(l.item())
            
        if (e+1) % 50 ==0:
            net.eval()
            with torch.no_grad():
                print('epoch %d , train loss mean : %f,train loss std : %f' % ( e+1,np.mean(l_sum),np.std(l_sum) ))

In [63]:
train_pytorch(1000,0.001,x9,y9,32)

epoch 50 , train loss mean : 0.029320,train loss std : 0.011825
epoch 100 , train loss mean : 0.013447,train loss std : 0.005459
epoch 150 , train loss mean : 0.009132,train loss std : 0.003357
epoch 200 , train loss mean : 0.005775,train loss std : 0.001916
epoch 250 , train loss mean : 0.005784,train loss std : 0.002395
epoch 300 , train loss mean : 0.004202,train loss std : 0.001604
epoch 350 , train loss mean : 0.003526,train loss std : 0.001398
epoch 400 , train loss mean : 0.002931,train loss std : 0.001270


KeyboardInterrupt: 

In [56]:
471*12

5652

### compared with linear model

In [77]:
def train_pytorch_LInear(num_epochs,lr,train_features,train_labels,batch_size):
    net=nn.Sequential(nn.Linear(train_features.shape[1],128),nn.ReLU(),nn.Linear(128,1))
    loss=nn.MSELoss()
    optimizer=optim.Adam(net.parameters(),lr=lr)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for e in range(num_epochs):
        l_sum=[]
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum.append(l.item())
        if (e+1) % 50 ==0:
            net.eval()
            with torch.no_grad():
                print('epoch %d , train loss mean : %f,train loss std : %f' % ( e+1,np.mean(l_sum),np.std(l_sum) ))

In [78]:
train_pytorch_LInear(1000,0.001,x9,y9,32)

epoch 50 , train loss mean : 0.038039,train loss std : 0.014225
epoch 100 , train loss mean : 0.025197,train loss std : 0.010772
epoch 150 , train loss mean : 0.014195,train loss std : 0.006786
epoch 200 , train loss mean : 0.010302,train loss std : 0.004486
epoch 250 , train loss mean : 0.009503,train loss std : 0.004339
epoch 300 , train loss mean : 0.008229,train loss std : 0.003656
epoch 350 , train loss mean : 0.007078,train loss std : 0.002974
epoch 400 , train loss mean : 0.007720,train loss std : 0.003838
epoch 450 , train loss mean : 0.005004,train loss std : 0.002256
epoch 500 , train loss mean : 0.005787,train loss std : 0.003748
epoch 550 , train loss mean : 0.004788,train loss std : 0.002233
epoch 600 , train loss mean : 0.004894,train loss std : 0.002861
epoch 650 , train loss mean : 0.004689,train loss std : 0.001956
epoch 700 , train loss mean : 0.004684,train loss std : 0.002623
epoch 750 , train loss mean : 0.006662,train loss std : 0.004670
epoch 800 , train loss mea